In [ ]:
from selenium.webdriver.common.by import By
from _nytimes import Puzzle

In [ ]:
browser = Puzzle("games/connections")

In [ ]:
words = [ card.text for card in browser.find_elements(By.XPATH, "//label[contains(@for, 'inner-card')]") ]

In [ ]:
browser.close()
browser.quit()